In [2]:
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pandas]


In [3]:
import pandas as pd

# Train
df = pd.read_csv("datasplit/train_metadata.csv")
df["file_path"] = df["file_path"].str.replace(
    "/content/drive/MyDrive/266/LJSpeech-1.1/wavs",
    "LJSpeech-1.1/wavs",
    regex=False
)
df.to_csv("datasplit/train_metadata.csv", index=False)

# Test
df = pd.read_csv("datasplit/test_metadata.csv")
df["file_path"] = df["file_path"].str.replace(
    "/content/drive/MyDrive/266/LJSpeech-1.1/wavs",
    "LJSpeech-1.1/wavs",
    regex=False
)
df.to_csv("datasplit/test_metadata.csv", index=False)

print("✔ Both manifests updated successfully!")


✔ Both manifests updated successfully!


In [4]:
!pip install safetensors librosa accelerate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.4/800.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19/19 [accelerate]


In [5]:
!python save_taco_mels.py \
    --path_to_manifest datasplit/train_metadata.csv datasplit/test_metadata.csv \
    --path_to_save data/taco_gen_mels \
    --taco_weights work_dir/taco_weights/taco_model.bin \
    --workers 0


Loaded Tacotron2 weights: <All keys matched successfully>
Processing manifest: datasplit/train_metadata.csv
100%|█████████████████████████████████████████| 405/405 [07:39<00:00,  1.14s/it]
Processing manifest: datasplit/test_metadata.csv
100%|█████████████████████████████████████████████| 5/5 [00:05<00:00,  1.11s/it]


In [6]:
!accelerate launch train.py \
    --experiment_name hifigan_finetune_fast \
    --working_directory work_dir \
    --path_to_train_manifest datasplit/train_metadata.csv \
    --path_to_val_manifest datasplit/test_metadata.csv \
    --path_to_saved_mels data/taco_gen_mels \
    --path_to_pretrained_weights work_dir/hifigan/final_checkpoint/model.safetensors \
    --finetune \
    --training_epochs 50 \
    --console_out_iters 20 \
    --wandb_log_iters 20 \
    --checkpoint_epochs 5 \
    --batch_size 16 \
    --learning_rate 0.00015 \
    --beta1 0.8 \
    --beta2 0.99 \
    --lr_decay 0.999 \
    --upsample_rates 8 8 2 2 \
    --upsample_kernel_sizes 16 16 4 4 \
    --resblock_kernel_sizes 3 7 11 \
    --resblock_dilation_sizes "((1,3,5),(1,3,5),(1,3,5))" \
    --mpd_periods 2 3 5 7 11 \
    --sampling_rate 22050 \
    --segment_size 8192 \
    --num_mels 80 \
    --n_fft 1024 \
    --window_size 1024 \
    --hop_size 256 \
    --fmin 0 \
    --fmax 8000 \
    --num_workers 2 \
    --log_wandb


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
/opt/conda/lib/python3.11/site-packages/accelerate/accelerator.py:529: UserWarning: `log_with=wandb` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")
Effective Upsample Rate: 256
Loading Pretrained HIFIGAN for Finetuning
Epoch: 0
Completed Steps: 0 | Gen Loss: 37.4659 | Disc Loss: 2.4471 | Mel Loss: 0.5517
Completed Steps: 20 | Gen Loss: 27.7999 | Disc Loss: 3.0916 | Mel Loss: 0.3718
Completed Steps: 40 | Gen Loss: 26.8232 | Disc Loss: 3.0346 | Mel Loss: 0.3596
Completed Steps: 60 | Gen Loss: 27